# 🎾RandomForest с уменьшенным числом деревьев и глубиной

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Функция для имитации реалистичных пропусков в теннисных данных
def create_realistic_tennis_missing_data(tennis_df, random_state=None):
    np.random.seed(random_state)
    df = tennis_df.copy()

    # 1. Рейтинг и очки чаще отсутствуют у малоизвестных игроков
    high_rank_mask = df['Rank_1'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_1'] = np.nan

    high_rank_mask = df['Rank_2'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_2'] = np.nan

    # 2. Коэффициенты могут отсутствовать для некоторых матчей
    df.loc[np.random.random(len(df)) < 0.1, ['Odd_1', 'Odd_2']] = np.nan

    # 3. Счет может быть неполным для незавершенных матчей
    # Имитируем отсутствие последних сетов
    def truncate_score(score, prob_missing=0.15):
        if pd.isna(score) or np.random.random() > prob_missing:
            return score

        sets = score.split()
        if len(sets) <= 1:
            return score

        # Сохраняем только первый сет с вероятностью 0.7, иначе первые два
        n_sets = 1 if np.random.random() < 0.7 else 2
        n_sets = min(n_sets, len(sets))

        return ' '.join(sets[:n_sets])

    df['Score'] = df['Score'].apply(lambda x: truncate_score(x))

    return df

# Функция для извлечения признаков из счета матча
def extract_score_features(score):
    if pd.isna(score):
        return [0] * 6
    
    sets = score.split()
    total_sets = len(sets)
    sets_won_1 = 0
    sets_won_2 = 0
    total_games_1 = 0
    total_games_2 = 0
    
    for set_score in sets:
        games = set_score.split('-')
        if len(games) == 2:
            games_1 = int(games[0])
            games_2 = int(games[1])
            total_games_1 += games_1
            total_games_2 += games_2
            if games_1 > games_2:
                sets_won_1 += 1
            else:
                sets_won_2 += 1
    
    return [total_sets, sets_won_1, sets_won_2, 
            total_games_1, total_games_2,
            total_games_1 + total_games_2]

# Функция для создания дополнительных признаков
def create_comparison_features(df):
    features = pd.DataFrame()
    
    # Рейтинговое преимущество (чем больше разница, тем сильнее преимущество)
    features['rank_advantage'] = (df['Rank_2'] - df['Rank_1']) / (df['Rank_1'] + df['Rank_2'])
    
    # Очковое преимущество
    features['points_advantage'] = (df['Pts_1'] - df['Pts_2']) / (df['Pts_1'] + df['Pts_2'])
    
    # Преимущество по коэффициентам
    features['odds_advantage'] = (df['Odd_2'] - df['Odd_1']) / (df['Odd_1'] + df['Odd_2'])
    
    # Бинарные признаки для прямого сравнения
    features['better_rank'] = (df['Rank_1'] < df['Rank_2']).astype(int)
    features['more_points'] = (df['Pts_1'] > df['Pts_2']).astype(int)
    features['better_odds'] = (df['Odd_1'] < df['Odd_2']).astype(int)
    
    # Составной показатель преимущества
    features['overall_advantage'] = (features['better_rank'] + 
                                   features['more_points'] + 
                                   features['better_odds']) / 3
    
    # Добавляем квадраты и взаимодействия признаков
    features['rank_points_interaction'] = features['rank_advantage'] * features['points_advantage']
    features['rank_odds_interaction'] = features['rank_advantage'] * features['odds_advantage']
    features['points_odds_interaction'] = features['points_advantage'] * features['odds_advantage']
    
    return features

# Обновленная функция подготовки данных
def prepare_data(df):
    df = df.copy()
    
    # Конвертируем дату в datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Заменяем отрицательные значения на NaN
    numeric_columns = ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']
    for col in numeric_columns:
        df.loc[df[col] < 0, col] = np.nan
    
    # Извлекаем признаки из счета
    score_features = df['Score'].apply(extract_score_features)
    score_features_df = pd.DataFrame(score_features.tolist(), 
                                   columns=['total_sets', 'sets_won_1', 'sets_won_2',
                                          'total_games_1', 'total_games_2', 'total_games'])
    
    # Создаем признак результата
    df['target'] = (df['Winner'] == df['Player_1']).astype(int)
    
    # Кодируем категориальные признаки
    le = LabelEncoder()
    categorical_columns = ['Series', 'Court', 'Surface', 'Round']
    for col in categorical_columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    # Добавляем новые признаки сравнения
    comparison_features = create_comparison_features(df)
    
    # Объединяем все признаки
    features = pd.concat([
        df[['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']],
        df[[col + '_encoded' for col in categorical_columns]],
        score_features_df,
        comparison_features
    ], axis=1)
    
    return features, df['target']

# Функция для обучения модели с настроенными параметрами
def train_model(df_train):
    X, y = prepare_data(df_train)
    
    # Заполняем пропущенные значения
    X = X.fillna(X.mean())
    
    # Создаем модель с уменьшенным числом деревьев и настроенными параметрами
    model = RandomForestClassifier(
        n_estimators=50,           # Уменьшаем количество деревьев
        max_depth=4,               # Ограничиваем глубину для избежания переобучения
        min_samples_split=10,       # Минимальное количество образцов для разделения
        min_samples_leaf=5,         # Минимальное количество образцов в листе
        random_state=32,
        class_weight='balanced'     # Балансируем веса классов
    )
    model.fit(X, y)
    
    return model

# Функции get_predictions, create_results_table и display_results остаются без изменений
def get_predictions(model, df_test):
    X_test, _ = prepare_data(df_test)
    X_test = X_test.fillna(X_test.mean())
    probabilities = model.predict_proba(X_test)
    return probabilities

# Функция для создания таблицы результатов
def create_results_table(df_test, predictions):
    results = pd.DataFrame({
        'Player_1': df_test['Player_1'],
        'Player_1_Prediction': predictions[:, 1] * 100,
        'Player_2': df_test['Player_2'],
        'Player_2_Prediction': predictions[:, 0] * 100,
        'Winner': df_test['Winner'],
        'Rank_1': df_test['Rank_1'],
        'Rank_2': df_test['Rank_2'],
        'Pts_1': df_test['Pts_1'],
        'Pts_2': df_test['Pts_2'],
        'Odd_1': df_test['Odd_1'],
        'Odd_2': df_test['Odd_2'],
        'Score': df_test['Score']
    })
    
    # Определяем, кого модель прогнозирует победителем
    results['predicted_winner'] = np.where(predictions[:, 1] > predictions[:, 0], 
                                         df_test['Player_1'], 
                                         df_test['Player_2'])
    
    # Определяем, было ли предсказание верным
    results['correct_prediction'] = (results['predicted_winner'] == df_test['Winner'])
    
    # Добавляем уверенность в прогнозе - максимальная из двух вероятностей
    results['confidence'] = np.maximum(predictions[:, 0], predictions[:, 1]) * 100
    
    return results

# Функция для отображения результатов
def display_results(results):
    styled_results = results.style.format({
        'Player_1_Prediction': '{:.2f}%',
        'Player_2_Prediction': '{:.2f}%',
        'Rank_1': '{:.0f}',
        'Rank_2': '{:.0f}',
        'Pts_1': '{:.0f}',
        'Pts_2': '{:.0f}',
        'Odd_1': '{:.2f}',
        'Odd_2': '{:.2f}'
    }).apply(lambda x: ['background-color: #90EE90' if x['correct_prediction'] 
                       else 'background-color: #FFB6C6' for i in range(len(x))], axis=1)
    
    display(styled_results.hide(axis='columns', subset=['correct_prediction']))

# Функция для оценки и вывода метрик
def display_metrics(model, df_test, predictions, results):
    # Истинные значения (1 - если победил первый игрок, 0 - если второй)
    y_true = (df_test['Winner'] == df_test['Player_1']).astype(int)
    
    # Предсказанные вероятности победы первого игрока
    y_pred_proba = predictions[:, 1]
    
    # Предсказанные бинарные метки
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Расчет метрик
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred) * 100
    roc_auc = roc_auc_score(y_true, y_pred_proba) * 100
    entropy_loss = log_loss(y_true, y_pred_proba)
    
    # Матрица ошибок
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Выводим метрики
    print(f"\nМетрики качества модели:")
    print(f"Accuracy (Точность): {accuracy:.2f}%")
    print(f"Precision (Точность по положительному классу): {precision:.2f}%")
    print(f"Recall (Полнота): {recall:.2f}%")
    print(f"F1-score (F-мера): {f1:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}%")
    print(f"Log Loss: {entropy_loss:.4f}")
    
    # Анализ ошибок
    print("\nМатрица ошибок (Confusion Matrix):")
    print(f"True Negative: {tn} (Правильно предсказанные победы второго игрока)")
    print(f"False Positive: {fp} (Неправильно предсказанные победы первого игрока)")
    print(f"False Negative: {fn} (Неправильно предсказанные победы второго игрока)")
    print(f"True Positive: {tp} (Правильно предсказанные победы первого игрока)")
    
    # Расчет метрик по уверенности прогноза
    confidence_bins = [50, 60, 70, 80, 90, 100]
    print("\nТочность предсказаний по уровню уверенности:")
    
    for i in range(len(confidence_bins)-1):
        low = confidence_bins[i]
        high = confidence_bins[i+1]
        
        # Фильтруем предсказания по уровню уверенности
        confident_preds = results[(results['confidence'] >= low) & (results['confidence'] < high)]
        
        if len(confident_preds) > 0:
            bin_accuracy = (confident_preds['correct_prediction'].sum() / len(confident_preds)) * 100
            print(f"Уверенность {low}%-{high}%: {bin_accuracy:.2f}% точность ({len(confident_preds)} матчей)")
        else:
            print(f"Уверенность {low}%-{high}%: нет матчей")
    
    # Выводим важность признаков
    feature_names = (
        ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2'] +
        ['Series_encoded', 'Court_encoded', 'Surface_encoded', 'Round_encoded'] +
        ['total_sets', 'sets_won_1', 'sets_won_2', 'total_games_1', 'total_games_2', 'total_games'] +
        ['rank_advantage', 'points_advantage', 'odds_advantage', 'better_rank', 'more_points', 'better_odds', 'overall_advantage', 
         'rank_points_interaction', 'rank_odds_interaction', 'points_odds_interaction']
    )
    
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nВажность признаков:")
    print(feature_importance.head(10))

---

## 1. Обучение модели на большой выборке без пропущенных значений

In [2]:
df_train = pd.read_csv('atp_tennis.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Время обучения: 1.77 секунд
Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [3]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,1.00%,Dolgopolov O.,99.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,98.995508
1,Harrison R.,47.51%,Sela D.,52.49%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Sela D.,52.485242
2,Anderson K.,51.26%,Edmund K.,48.74%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,51.256194
3,Carreno Busta P.,92.59%,Kubler J.,7.41%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,92.585531
4,Youzhny M.,0.63%,Cuevas P.,99.37%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.367601
5,Istomin D.,98.34%,Herbert P.H.,1.66%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,98.344577
6,Delbonis F.,0.87%,Muller G.,99.13%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.132255
7,Seppi A.,96.43%,Moutet C.,3.57%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,96.428677
8,Ferrer D.,36.79%,Rublev A.,63.21%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,63.212401
9,Ebden M.,87.67%,Isner J.,12.33%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,87.669601



Метрики качества модели:
Accuracy (Точность): 93.50%
Precision (Точность по положительному классу): 94.92%
Recall (Полнота): 91.80%
F1-score (F-мера): 93.33%
ROC AUC: 98.97%
Log Loss: 0.1468

Матрица ошибок (Confusion Matrix):
True Negative: 59 (Правильно предсказанные победы второго игрока)
False Positive: 3 (Неправильно предсказанные победы первого игрока)
False Negative: 5 (Неправильно предсказанные победы второго игрока)
True Positive: 56 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 58.82% точность (17 матчей)
Уверенность 60%-70%: 83.33% точность (6 матчей)
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: 100.00% точность (9 матчей)
Уверенность 90%-100%: 100.00% точность (91 матчей)

Важность признаков:
           Feature  Importance
12      sets_won_2    0.368582
11      sets_won_1    0.299726
14   total_games_2    0.130149
13   total_games_1    0.099899
4            Odd_1    0.017908
15     total_games    

### Тестирование на выборке с пропущенными значениями

In [4]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,31.49%,Dolgopolov O.,68.51%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,68.506624
1,Harrison R.,47.51%,Sela D.,52.49%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Sela D.,52.485242
2,Anderson K.,51.26%,Edmund K.,48.74%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,51.256194
3,Carreno Busta P.,92.59%,Kubler J.,7.41%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,92.585531
4,Youzhny M.,0.63%,Cuevas P.,99.37%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.367601
5,Istomin D.,98.34%,Herbert P.H.,1.66%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,98.344577
6,Delbonis F.,0.87%,Muller G.,99.13%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.132255
7,Seppi A.,47.29%,Moutet C.,52.71%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,52.712209
8,Ferrer D.,36.79%,Rublev A.,63.21%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,63.212401
9,Ebden M.,58.39%,Isner J.,41.61%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,58.390200



Метрики качества модели:
Accuracy (Точность): 91.06%
Precision (Точность по положительному классу): 93.10%
Recall (Полнота): 88.52%
F1-score (F-мера): 90.76%
ROC AUC: 98.55%
Log Loss: 0.1753

Матрица ошибок (Confusion Matrix):
True Negative: 58 (Правильно предсказанные победы второго игрока)
False Positive: 4 (Неправильно предсказанные победы первого игрока)
False Negative: 7 (Неправильно предсказанные победы второго игрока)
True Positive: 54 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 52.38% точность (21 матчей)
Уверенность 60%-70%: 80.00% точность (5 матчей)
Уверенность 70%-80%: 100.00% точность (5 матчей)
Уверенность 80%-90%: 100.00% точность (8 матчей)
Уверенность 90%-100%: 100.00% точность (84 матчей)

Важность признаков:
           Feature  Importance
12      sets_won_2    0.368582
11      sets_won_1    0.299726
14   total_games_2    0.130149
13   total_games_1    0.099899
4            Odd_1    0.017908
15   

---

## 2. Обучение модели на большой выборке с пропущенными значениями

In [5]:
df_train = pd.read_csv('atp_tennis.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [6]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.98%,Dolgopolov O.,99.02%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.021103
1,Harrison R.,44.89%,Sela D.,55.11%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Sela D.,55.110908
2,Anderson K.,51.39%,Edmund K.,48.61%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,51.388222
3,Carreno Busta P.,92.97%,Kubler J.,7.03%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,92.968059
4,Youzhny M.,1.62%,Cuevas P.,98.38%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,98.380640
5,Istomin D.,96.44%,Herbert P.H.,3.56%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,96.435167
6,Delbonis F.,1.25%,Muller G.,98.75%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,98.750938
7,Seppi A.,93.21%,Moutet C.,6.79%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,93.206577
8,Ferrer D.,44.85%,Rublev A.,55.15%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,55.148812
9,Ebden M.,82.40%,Isner J.,17.60%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,82.398785



Метрики качества модели:
Accuracy (Точность): 94.31%
Precision (Точность по положительному классу): 96.55%
Recall (Полнота): 91.80%
F1-score (F-мера): 94.12%
ROC AUC: 98.89%
Log Loss: 0.1683

Матрица ошибок (Confusion Matrix):
True Negative: 60 (Правильно предсказанные победы второго игрока)
False Positive: 2 (Неправильно предсказанные победы первого игрока)
False Negative: 5 (Неправильно предсказанные победы второго игрока)
True Positive: 56 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 60.00% точность (15 матчей)
Уверенность 60%-70%: 87.50% точность (8 матчей)
Уверенность 70%-80%: 100.00% точность (1 матчей)
Уверенность 80%-90%: 100.00% точность (15 матчей)
Уверенность 90%-100%: 100.00% точность (84 матчей)

Важность признаков:
              Feature  Importance
12         sets_won_2    0.307630
11         sets_won_1    0.280447
14      total_games_2    0.132568
13      total_games_1    0.131437
10         total_set

### Тестирование на выборке с пропущенными значениями

In [7]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,18.78%,Dolgopolov O.,81.22%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,81.223161
1,Harrison R.,44.89%,Sela D.,55.11%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Sela D.,55.110908
2,Anderson K.,51.39%,Edmund K.,48.61%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,51.388222
3,Carreno Busta P.,92.97%,Kubler J.,7.03%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,92.968059
4,Youzhny M.,1.62%,Cuevas P.,98.38%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,98.380640
5,Istomin D.,96.44%,Herbert P.H.,3.56%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,96.435167
6,Delbonis F.,1.25%,Muller G.,98.75%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,98.750938
7,Seppi A.,49.37%,Moutet C.,50.63%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,50.633867
8,Ferrer D.,44.85%,Rublev A.,55.15%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,55.148812
9,Ebden M.,47.41%,Isner J.,52.59%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,52.593920



Метрики качества модели:
Accuracy (Точность): 91.87%
Precision (Точность по положительному классу): 94.74%
Recall (Полнота): 88.52%
F1-score (F-мера): 91.53%
ROC AUC: 98.33%
Log Loss: 0.1921

Матрица ошибок (Confusion Matrix):
True Negative: 59 (Правильно предсказанные победы второго игрока)
False Positive: 3 (Неправильно предсказанные победы первого игрока)
False Negative: 7 (Неправильно предсказанные победы второго игрока)
True Positive: 54 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 52.63% точность (19 матчей)
Уверенность 60%-70%: 85.71% точность (7 матчей)
Уверенность 70%-80%: 100.00% точность (1 матчей)
Уверенность 80%-90%: 100.00% точность (21 матчей)
Уверенность 90%-100%: 100.00% точность (75 матчей)

Важность признаков:
              Feature  Importance
12         sets_won_2    0.307630
11         sets_won_1    0.280447
14      total_games_2    0.132568
13      total_games_1    0.131437
10         total_set

---

## 3. Обучение модели на маленькой выборке без пропущенных значений

In [8]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 50


### Тестирование на выборке без пропущенных значений

In [9]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,5.85%,Dolgopolov O.,94.15%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,94.147186
1,Harrison R.,50.70%,Sela D.,49.30%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,50.696293
2,Anderson K.,54.96%,Edmund K.,45.04%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,54.963478
3,Carreno Busta P.,81.60%,Kubler J.,18.40%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,81.604343
4,Youzhny M.,17.88%,Cuevas P.,82.12%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,82.120879
5,Istomin D.,78.70%,Herbert P.H.,21.30%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,78.696032
6,Delbonis F.,8.84%,Muller G.,91.16%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,91.156161
7,Seppi A.,74.08%,Moutet C.,25.92%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,74.083189
8,Ferrer D.,47.17%,Rublev A.,52.83%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,52.834682
9,Ebden M.,38.32%,Isner J.,61.68%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Isner J.,61.675758



Метрики качества модели:
Accuracy (Точность): 88.62%
Precision (Точность по положительному классу): 89.83%
Recall (Полнота): 86.89%
F1-score (F-мера): 88.33%
ROC AUC: 97.58%
Log Loss: 0.3268

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 8 (Неправильно предсказанные победы второго игрока)
True Positive: 53 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 37.50% точность (16 матчей)
Уверенность 60%-70%: 84.21% точность (19 матчей)
Уверенность 70%-80%: 97.30% точность (37 матчей)
Уверенность 80%-90%: 100.00% точность (36 матчей)
Уверенность 90%-100%: 100.00% точность (15 матчей)

Важность признаков:
              Feature  Importance
11         sets_won_1    0.268420
12         sets_won_2    0.170194
18     odds_advantage    0.136035
4               Odd_1    0.098840
14      total_games

### Тестирование на выборке с пропущенными значениями

In [10]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,23.85%,Dolgopolov O.,76.15%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,76.147186
1,Harrison R.,50.70%,Sela D.,49.30%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,50.696293
2,Anderson K.,54.96%,Edmund K.,45.04%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,54.963478
3,Carreno Busta P.,81.60%,Kubler J.,18.40%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,81.604343
4,Youzhny M.,17.88%,Cuevas P.,82.12%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,82.120879
5,Istomin D.,78.70%,Herbert P.H.,21.30%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,78.696032
6,Delbonis F.,8.84%,Muller G.,91.16%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,91.156161
7,Seppi A.,51.91%,Moutet C.,48.09%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Seppi A.,51.905411
8,Ferrer D.,47.17%,Rublev A.,52.83%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,52.834682
9,Ebden M.,25.23%,Isner J.,74.77%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,74.773810



Метрики качества модели:
Accuracy (Точность): 86.99%
Precision (Точность по положительному классу): 88.14%
Recall (Полнота): 85.25%
F1-score (F-мера): 86.67%
ROC AUC: 96.39%
Log Loss: 0.3572

Матрица ошибок (Confusion Matrix):
True Negative: 55 (Правильно предсказанные победы второго игрока)
False Positive: 7 (Неправильно предсказанные победы первого игрока)
False Negative: 9 (Неправильно предсказанные победы второго игрока)
True Positive: 52 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 38.10% точность (21 матчей)
Уверенность 60%-70%: 95.24% точность (21 матчей)
Уверенность 70%-80%: 94.59% точность (37 матчей)
Уверенность 80%-90%: 100.00% точность (33 матчей)
Уверенность 90%-100%: 100.00% точность (11 матчей)

Важность признаков:
              Feature  Importance
11         sets_won_1    0.268420
12         sets_won_2    0.170194
18     odds_advantage    0.136035
4               Odd_1    0.098840
14      total_games

---

## 4. Обучение модели на маленькой выборке с пропущенными значениями

In [11]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

Размер тренировочного набора: 50


### Тестирование на выборке без пропущенных значений

In [12]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,4.37%,Dolgopolov O.,95.63%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,95.634848
1,Harrison R.,64.14%,Sela D.,35.86%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,64.138764
2,Anderson K.,63.04%,Edmund K.,36.96%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,63.035711
3,Carreno Busta P.,70.92%,Kubler J.,29.08%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,70.917457
4,Youzhny M.,14.50%,Cuevas P.,85.50%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,85.502839
5,Istomin D.,69.03%,Herbert P.H.,30.97%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,69.031602
6,Delbonis F.,9.41%,Muller G.,90.59%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,90.590801
7,Seppi A.,62.80%,Moutet C.,37.20%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,62.801751
8,Ferrer D.,58.73%,Rublev A.,41.27%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,58.726107
9,Ebden M.,28.18%,Isner J.,71.82%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Isner J.,71.822294



Метрики качества модели:
Accuracy (Точность): 91.06%
Precision (Точность по положительному классу): 90.32%
Recall (Полнота): 91.80%
F1-score (F-мера): 91.06%
ROC AUC: 96.01%
Log Loss: 0.3715

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 5 (Неправильно предсказанные победы второго игрока)
True Positive: 56 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 81.25% точность (16 матчей)
Уверенность 60%-70%: 81.82% точность (33 матчей)
Уверенность 70%-80%: 95.12% точность (41 матчей)
Уверенность 80%-90%: 100.00% точность (20 матчей)
Уверенность 90%-100%: 100.00% точность (13 матчей)

Важность признаков:
              Feature  Importance
11         sets_won_1    0.247421
12         sets_won_2    0.166280
18     odds_advantage    0.090889
22  overall_advantage    0.074662
13      total_games

### Тестирование на выборке с пропущенными значениями

In [13]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,12.73%,Dolgopolov O.,87.27%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,87.268182
1,Harrison R.,64.14%,Sela D.,35.86%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,64.138764
2,Anderson K.,63.04%,Edmund K.,36.96%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,63.035711
3,Carreno Busta P.,70.92%,Kubler J.,29.08%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,70.917457
4,Youzhny M.,14.50%,Cuevas P.,85.50%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,85.502839
5,Istomin D.,69.03%,Herbert P.H.,30.97%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,69.031602
6,Delbonis F.,9.41%,Muller G.,90.59%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,90.590801
7,Seppi A.,46.38%,Moutet C.,53.62%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,53.624884
8,Ferrer D.,58.73%,Rublev A.,41.27%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,58.726107
9,Ebden M.,26.71%,Isner J.,73.29%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,73.286436



Метрики качества модели:
Accuracy (Точность): 90.24%
Precision (Точность по положительному классу): 90.16%
Recall (Полнота): 90.16%
F1-score (F-мера): 90.16%
ROC AUC: 95.61%
Log Loss: 0.3782

Матрица ошибок (Confusion Matrix):
True Negative: 56 (Правильно предсказанные победы второго игрока)
False Positive: 6 (Неправильно предсказанные победы первого игрока)
False Negative: 6 (Неправильно предсказанные победы второго игрока)
True Positive: 55 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 77.78% точность (18 матчей)
Уверенность 60%-70%: 80.00% точность (30 матчей)
Уверенность 70%-80%: 95.00% точность (40 матчей)
Уверенность 80%-90%: 100.00% точность (25 матчей)
Уверенность 90%-100%: 100.00% точность (10 матчей)

Важность признаков:
              Feature  Importance
11         sets_won_1    0.247421
12         sets_won_2    0.166280
18     odds_advantage    0.090889
22  overall_advantage    0.074662
13      total_games